In [5]:
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torchvision

import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime
import os

# Constants

In [6]:
# dataset path
DATA_DF_PATH = './dataset/'
DATA_IMG_PATH = '../dataset/kaggle_rsna(only100)/'

# hyper parameter
MODEL_NAME = 'DenseNet121'
LOSS_NAME = 'BCE'
EPOCHS = 100
BATCH_SIZE = 64
INITIAL_LR = 0.001

# etc path
MODEL_WEIGHTS_SAVE_PATH = './checkpoints/'
MODEL_WEIGHTS_LOAD_PATH = './checkpoints/'
TENSORBOARD_PATH = './tensorboard/'

# gpu settings
IS_GPU_PARALLEL = True if torch.cuda.device_count()>1 else False

# train id
timestamp = datetime.now().strftime('%y%m%d_%H%M%S')
TRAIN_ID = f'{timestamp}_{MODEL_NAME}_LR{INITIAL_LR}_BS{BATCH_SIZE}_{LOSS_NAME}Loss'

# dataset mode
DATASET_MODE = 'sequential' # '3ch' or 'sequential'

# 모델 생성

In [7]:
class DenseNet121_change_avg(nn.Module):
    def __init__(self):
        super(DenseNet121_change_avg, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True).features
        self.avgpool = nn.AdaptiveAvgPool2d(1)  
        self.relu = nn.ReLU()
        self.mlp = nn.Linear(1024, 6)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.densenet121(x)      
        x = self.relu(x)
        x = self.avgpool(x)
        x_features = x.view(-1, 1024)
        x = self.mlp(x_features)
        x = self.sigmoid(x)
        
        return x, x_features
    
model = DenseNet121_change_avg()

In [8]:
from torchsummary import summary
model = DenseNet121_change_avg()
# summary(model, (3,512,512), device='cpu')

# Transform 전처리기 생성

In [9]:
import torchvision.transforms as T
transforms = T.Compose([T.Resize([256,256]),
                        T.ToTensor()])

# 데이터셋 세팅

In [10]:
class HmDataset(Dataset):
    def __init__(self, df_path, transforms=None, mode='3ch'):
        self.df = pd.read_csv(df_path)
        self.transforms = transforms
        self.mode = mode
        
    def __getitem__(self, index):
        hm_meta = self.df.iloc[index]
        filename = hm_meta.filename
        label = torch.from_numpy(hm_meta['epidural':'any'].values.astype(np.float))
        
        img = Image.open('../dataset/kaggle_rsna(only100)/imgs/'+filename+'.png')
        
        if self.mode=='sequential':
            study_instance_uid = hm_meta.study_instance_uid
            slice_id_current = hm_meta.slice_id
            
            # 해당 환자의 DataFrame
            tmp_df = self.df[self.df.study_instance_uid==study_instance_uid]
            
            # 해당 환자의 slice 전체 수 
            max_slice = tmp_df.shape[0]
            
            # get prev, next slice number
            slice_id_prev = slice_id_current if slice_id_current==0 else slice_id_current-1
            slice_id_next = slice_id_current if slice_id_current==max_slice-1 else slice_id_current+1
            
            # get prev, next filename
            filename_prev = tmp_df[tmp_df.slice_id==slice_id_prev].iloc[0].filename
            filename_next = tmp_df[tmp_df.slice_id==slice_id_next].iloc[0].filename
            
            # get prev, next img
            img_prev = Image.open('../dataset/kaggle_rsna(only100)/imgs/'+filename_prev+'.png')
            img_next = Image.open('../dataset/kaggle_rsna(only100)/imgs/'+filename_next+'.png')
            
            # concat 3 imgs
            img = np.concatenate([np.expand_dims(np.array(img)[:,:,0], axis=2),
                                   np.expand_dims(np.array(img_prev)[:,:,0], axis=2),
                                   np.expand_dims(np.array(img_next)[:,:,0], axis=2)],
                                  axis=2)
            img = Image.fromarray(img)
                                   

        if self.transforms is not None:
            img = self.transforms(img)
        
        return filename, label, img
    
    def __len__(self):
        return len(self.df)

train_dataset = HmDataset(df_path='./dataset/train.csv', transforms=transforms, mode=DATASET_MODE)
valid_dataset = HmDataset(df_path='./dataset/valid.csv', transforms=transforms, mode=DATASET_MODE)
test_dataset = HmDataset(df_path='./dataset/test.csv', transforms=transforms, mode=DATASET_MODE)

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=4)
valid_loader = DataLoader(valid_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4)

# 학습

In [12]:
model =  DenseNet121_change_avg()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=0.9)

In [13]:
def fit(phase, epoch, model, data_loader, optimizer, criterion, device):
    
    losses = 0
    if phase=='Train':
        model.train()
    elif phase=='Valid' or phase=='Test':
        model.eval()
    
    tbar = tqdm(data_loader, position=0, leave=True)
    for data in tbar:
        
        _, target, input_img = data
        target, input_img = target.to(device), input_img.to(device)
        
        optimizer.zero_grad()
        
        predicted_label, _ = model(input_img)
        loss = criterion(predicted_label, target.float())
        
        if phase=='Train':
            loss.backward()
            optimizer.step()
        
#         predicted_label_thresholded = predicted_label>0.5
#         acc = (predicted_label_thresholded==target).sum() # 

        losses += loss.item()
        
        tbar.set_description(f'[{phase}]\tEpoch:[{epoch}/{EPOCHS}]\tLoss:{losses/len(data_loader):.5f}')# '\tAcc:{acc:.2%}')
        
    return losses/len(data_loader)
        

In [ ]:
# tensorboard log
writer = SummaryWriter(log_dir=os.path.join(TENSORBOARD_PATH, TRAIN_ID))
train_losses = []
valid_losses = []

for epoch in range(1, EPOCHS+1):
        
    # fit
    train_loss = fit('Train', epoch, model, train_loader, optimizer, criterion, device)
    with torch.no_grad():
        valid_loss = fit('Valid', epoch, model, valid_loader, optimizer, criterion, device)
        
    # log
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    # tensor board
    writer.add_scalar('Loss/Train/', train_loss, epoch)
    writer.add_scalar('Loss/Valid/', valid_loss, epoch)
#     writer.add_scalar('Accuracy/Train/', accuracy, epoch+1)

    # save model
    save_checkpoint(epoch, model, optimizer)
    
    if epoch%10==0:
        send_mail(f'[Epoch:{epoch}]학습 진행중', '')
    
send_mail(f'[알림]학습완료','EC2 종료할 것!!')



  0%|          | 0/6 [00:00<?, ?it/s]

In [1]:
def save_checkpoint(epoch, model, optimizer, scheduler=None):
    
    save_folder_dir = os.path.join(MODEL_WEIGHTS_SAVE_PATH, TRAIN_ID)
    if not os.path.exists(save_folder_dir):
        os.makedirs(save_folder_dir, exist_ok=True)
    model_save_path = os.path.join(save_folder_dir, f'{epoch:03d}.pth')
    
    if IS_GPU_PARALLEL:
        model_state_dict = model.module.state_dict()
    else:
        model_state_dict = model.state_dict()
        
    if scheduler is not None:
        scheduler = scheduler.state_dict()
        
    torch.save({
        'epoch':epoch,
        'model_state_dict':model_state_dict,
        'optimizer':optimizer.state_dict(),
        'scheduler':scheduler
    }, model_save_path)

In [2]:
def load_checkpoint(ckpt_path, model, optimizer=None, scheduler=None):
    
    if not os.path.exists(ckpt_path):
        raise ValueError('No ckpt in [{}]'.format(ckpt_path))
        
    ckpt = torch.load(ckpt_path)
    epoch = ckpt['epoch']
    model.load_state_dict(ckpt['model_state_dict'])
    
    if scheduler is not None:
        optimizer.load_state_dict(ckpt['optimizer'])

    if scheduler is not None:
        scheduler.load_state_dict(ckpt['scheduler'])
    
    return model, optimizer, scheduler, epoch

In [3]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_mail(subject, contents):
    # 세션생성, 로그인
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()  # TLS 사용시 필요
    s.login('jmjeon3155@gmail.com', 'simyiexludrwsxwn')

    # 제목, 본문 작성
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg.attach(MIMEText(contents, 'plain'))

    # 메일 전송
    s.sendmail("jmjeon3155@gmail.com", "jmjeon3155@gmail.com", msg.as_string())
    s.quit()